<h1><center> SEN163A - Fundamentals of Data Analytics </center>
<center> Assignment 2 - Large Scale Internet Data Analysis  </center></h1>

<center> Lecture:
Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl) </center>

## Group 3
- Hilmy Hanif - 5270243
- Terence Celestina - 1524658
- Leopoldo Ochoa Diaz - 5256992
- Ivy de Jong - 4932935

## Outline:
1. [Introduction](#1.-Introduction)
2. [Data Description](#2.-Data Description)
3. [Formulation](#3.-Formulation)
4. [Conclusion](#4.-Conclusion)
5. [Reference](#5.-Reference)

## 1. Introduction

### Definition

An autonomous system number is a unique identifier that is globally available and allows its autonomous system to exchange routing information with other systems.

### Overview

The Groote Nationale Investeer Bank (GNI Bank) is a large European bank and is preparing to enter the
mobile banking sector. They have hired your advisory firm to prepare a strategy. Your data analytics team
has been put on the task of finding the best four datacenter locations for hosting in the EU.


### Requested tasks:
- Evaluate if there are limitations in the provided datasets (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.
- With the AS and probe data set, find the number m of AS’s that can be used for hosting in the EU and have probes in the RIPE data set. Sort the ASN’s in ascending order and include the first and last three in your report (number, name and country).
- For a single hour in the RIPE data set: find all valid entries where the probe has hosting type AS and the target IPv4 is from an EU country. Implement this in an efficient way.
- Move from using only an hour to the full day. It is advisable to store the raw results of each file. Then, using all processed files, calculate the average latency’s for each country-AS combination and store the results into one ncountries ×m matrix. If we could place one server in each country, what would the minimum average latency be for each country? (include in your report)
- Since we are only allowed to place four servers, determine the best four datacenters based on the total latency for all countries. Report your findings and your procedure to obtain them. Also include the average latency for each country.


## 2. Data Description

### Task A
Evaluate AS data and Probe Data

## AS Data

In [52]:
import pickle
import pandas as pd
import ipaddress
import numpy as np

# Open the file in binary mode
with open('AS_dataset.pkl', 'rb') as file: #rb means in read mode
	
	# Call load method to deserialze
	AS_df = pickle.load(file)

In [53]:
AS_df.shape #know columns and rows of the AS dataframe

In [54]:
AS_df.head(10) #show first 10 row of the dataset

In [55]:
AS_df.tail(10) #show last 10 row of the dataset

### AS Column Description

- ASN : autonomus system number
- Country : specific country that is represented by the ASN
- Name : name of the company that use the ASN
- NumIPs : number of IPs used for the ASN
- Type : type of purpose of the ASN

In [56]:
AS_df.describe() #summarize description of the AS dataframe

From the table above, the NumIPs = 0 probably indicates that specified ASN is not used. In the next step, the NumIPs = 0 could be filtered from the data in order to use active ASN only.

In [57]:
AS_df[AS_df['NumIPs'] == '0'].count() / AS_df['NumIPs'].count() 
#identify NumIPs that is equal to 0, which is not utilized.

There are 25,12% of 0 NumIPs in the AS dataset. Take a note that later the NumIPs 0 should be filtered later to use only valid values.

In [58]:
AS_df['Country'].value_counts() #count total country in the dataset

There are 237 countries ranging from different continent. However this dataset only represents the country, not including the continent. To observe EU only, there should be another additional data point used to utilized only ASN from EU countries.

In [59]:
AS_df['type'].value_counts() #count value of type

In [60]:
AS_df[AS_df['NumIPs'] != '0']['type'].value_counts() #identify hosting type with NumIPs is not 0.

There are 5 types of the ASN purpose. Later, it is obligated to filter the dataset only for 'hosting' type which consists of 2310 data.

In [61]:
AS_df.info() 
#limitation: different data types between AS and Probe data should be converted into same data type. Data from pickle should be converted first
#format pkl data type provides all same object datatype

In [62]:
AS_df = AS_df.astype(dtype={'ASN': 'string'}) #convert the data type of ASN from object to string

In [63]:
AS_df['NumIPs'] = AS_df['NumIPs'].str.replace(',', '').astype(float).astype(int)
#remove comma between numbers
#convert to float
#convert to int

In [64]:
AS_df.info()

In [65]:
AS_df.isnull().any() #find if there are any null data

There are no null data for this AS dataset

## Probe Data

In [66]:
# Open the file in binary mode
with open('probe_dataset.pkl', 'rb') as file:
	
	# Call load method to deserialze
	probe_df = pickle.load(file)

In [67]:
probe_df.shape #identify row and column of the Probe Data. There are 2 columns and 11008 rows.

In [68]:
probe_df.head(10) #show first 10 data of Probe Data

### Probe Data Column Description

- prb_id : probe id. The probe id of the probe data is correspond to ASN. A probe is located in ASN which ASN contains information of the localization and the type.
- ASN : autonomus system number

In [69]:
probe_df = probe_df.astype(dtype={'ASN': 'string'})
probe_df = probe_df.astype(dtype={'prb_id': 'int'})
#convert the ASN and prb_id data type to string and integer

In [70]:
probe_df.info() #the datatype of prb_id and ASN is changed.

In [71]:
probe_df.describe() #describe the prb_id dataset

Unpickling: It is the inverse of Pickling process where a byte stream is converted into an object hierarchy. Therefore, the IP numbers is in integer. (?)

## RIPE Data (for one hour)

In [72]:
import os
import bz2
import time
import json
import glob
pd.set_option('display.max_columns', None)

In [73]:
#temp = pd.DataFrame()

#path_to_json ='https://data-store.ripe.net/datasets/atlas-daily-dumps/2022-03-01/ping-2022-03-01T0000' 

#json_pattern = os.path.join(path_to_json,'*.bz2')
#file_list = glob.glob(json_pattern)

#for file in file_list:
#    chunks = pd.read_json(file, lines=True, chunksize=1000)
#    i = 0
#    chunk_list = []
#    for chunk in chunks:
#        if i >= 10:
#            break
#        i += 1
#        chunk_list.append(chunk)
#        df = pd.concat(chunk_list, sort = True)
#    temp = temp.append(df, sort = True)
#temp

In [79]:
bz2Filename = '../ping-2022-03-01T0000.bz2'
bz2File     = bz2.open(bz2Filename, 'rt') 

# List of tuples
# https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop
tuple_list = []
#read first 10 lines to estimate total loading time
totalNrOfLines = 0;
filteredNrOfLines = 0

st  = time.time()
for line in bz2File:
    #print(line)
    decoded_line = json.loads(line)
    if(decoded_line["af"] == 4):
        filteredNrOfLines += 1
        tuple_list.append((decoded_line["dst_addr"],decoded_line["prb_id"],decoded_line["avg"]))
    #print(decoded_line)
    totalNrOfLines += 1
    if totalNrOfLines>1000: break

#print time and estimate total time            
dur         = round(time.time() - st,2)
estTotTime  = round( (dur/100000)*totalNrOfLines )
print("\nbz2 file:" )
print("Loading 1k lines took: "  + str(dur) + " seconds")
print("Estimated loading time of entire bz2 file: "  + str(estTotTime) + \
      " seconds" )

#finally close bz2File
bz2File.close()

The differences of dst_name and dst_address:
Dst_address: IP address of the destination (string)
Dst_name: name of the destination (string), could be the same as dst_address if it is not hosted yet, but the dst_name usually utilized as example: google.com, facebook.comw which actually represent the IP address.

### IPv4 data

This site or product includes IP2Location LITE data available from <a href="https://lite.ip2location.com">https://lite.ip2location.com</a>.

List of EU countries (see reference):
https://european-union.europa.eu/principles-countries-history/country-profiles_en?page=1

In [80]:
ipv4_df = pd.read_csv("IP2LOCATION-LITE-DB1.CSV")

In [81]:
ipv4_df.shape #identify total rows and columns in IPv4 data

In [82]:
ipv4_df.head() #display first 5 data of ipv4 data

In [83]:
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True) #add column name

In [84]:
ipv4_df.head() #display data with column name

### IPv4 Data Column Description

- ip_from: lowest range of the IP in the country
- ip_to: highest range of the IP in the country
- country_code: code of country with range of IP's
- country_name: name of country with range of IP's

In [85]:
ipv4_df.info() #collect information of the dataset

### Task B
Evaluate the number m of AS’s that can be used for hosting in the EU
and have probes in the RIPE data set.

In [86]:
#merge the AS dataset and Probe data
AS_pb_joined = probe_df.merge(AS_df, left_on='ASN', right_on='ASN', how='inner')
AS_pb_joined = AS_pb_joined.replace(np.nan,'', regex=True) 

In [87]:
AS_pb_joined.tail() #show last 5 of the merged dataset

In [88]:
AS_pb_joined.info() #identify the datatype of the merged dataset

In [89]:
print(AS_pb_joined[AS_pb_joined.prb_id == AS_pb_joined.prb_id.max()]) #identify max value of the prb_id

In [409]:
#add new column for determining EU or non-EU
# read text file
my_file = open("EU_country.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
EU_country_list = data.split("\n")
print(EU_country_list)
my_file.close()

['AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR', 'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL', 'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE']


In [91]:
#filter hosting type of AS_pb_joined
AS_pb_joined = AS_pb_joined[AS_pb_joined['type'] == 'hosting']

In [92]:
#filter the AS_pb_joined only to EU country
EU_AS_pb_joined = AS_pb_joined[AS_pb_joined.Country.isin(EU_country_list)]

In [93]:
#show 10 first dataset of EU_AS_pb_joined
EU_AS_pb_joined.head(10)

In [94]:
#describe the dataset
EU_AS_pb_joined.describe

In [95]:
EU_AS_pb_joined[EU_AS_pb_joined['NumIPs'] == '0'].count()  #identify row with NumIPs = 0

There are 339 rows which filtered by:
Type = Hosting;
Country = in Europe;
And within the dataset, the NumIPs = 0 is already excluded.

In [96]:
#sort the ASN's in ascending order 
sorted_df = EU_AS_pb_joined.sort_values('ASN', key=lambda x: x.str.extract('(\d+)').squeeze().astype(int))
print(sorted_df)

In [97]:
#show 3 first data of the sorted dataset
sorted_df[['ASN', 'Name', 'Country']].head(3)

In [98]:
#show 3 last data of the sorted dataset
sorted_df[['ASN', 'Name', 'Country']].tail(3)

In [99]:
unique_sorted_df = sorted_df.ASN.unique()
len(unique_sorted_df)

There are 145 unique ASN from 339 rows of sorted dataset.

### Task C
For a single hour in the RIPE data set: find all valid entries where the probe has hosting type AS and
the target IPv4 is from an EU country. Implement this in an efficient way

In [100]:
#store the timestamp to integer (or convert as you want as required)

In [130]:
prb_id_values=list(set(sorted_df.prb_id.values))

In [101]:
#decoded_line = filter(lambda decoded_line: decoded_line['mver'], decoded_line)

In [ ]:
ip_ranges_all = list(zip(EUip_df.ip_from, EUip_df.ip_to))

In [253]:
EU_ipv4_df = ipv4_df[ipv4_df.country_code.isin(EU_country_list)]

#Make a zip list of IP addresses within the EU:
ip_eu = list(zip(EU_ipv4_df.ip_from, EU_ipv4_df.ip_to))

In [ ]:
#Adapted a version of this C++ https://stackoverflow.com/questions/22370851/binary-search-for-a-range.
#This function should help me only pick up IP addresses that are between low and high (being IP ranges)

In [274]:
def ip_exist_eu(ip, ip_eu): 
    
    low, high = 0, len(ip_eu)
    
    for n in range(len(ip_eu)):
        mid = low + (high - low) // 2
        
        if value <= ip_eu[mid - 1][1]: 
            right = mid
        elif value >= ip_eu[mid][0]:
            left = mid
        else: return None

    if ip[left][0]<=value<=ip[left][1]:
        return True
    else:
        return False
            

In [413]:
import ipaddress
bz2Filename = '../ping-2022-03-01T0000.bz2'
bz2File     = bz2.open(bz2Filename, 'rt') 

# List of tuples
# https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop
tuple_list = []
probes_in_RIPE_list = []
#read first 10 lines to estimate total loading time
totalNrOfLines = 0
filteredNrOfLines = 0

start  = time.time()
for line in bz2File:
    #print(line)
    decoded_line = json.loads(line)
    if "af" in decoded_line and "dst_addr" in decoded_line and "prb_id" in decoded_line and "avg":
        #added extra logic in this IF statement to apply the function I defined above (for belonging to EU or not)
        if(decoded_line["af"] == 4 and (decoded_line["prb_id"] in sorted_df["prb_id"]) and (decoded_line["avg"] > 0)):
            filteredNrOfLines += 1
            probes_in_RIPE_list.append(decoded_line["prb_id"])
            tuple_list.append((decoded_line["dst_addr"],decoded_line["prb_id"],decoded_line["avg"]))
    #print(decoded_line)
    totalNrOfLines += 1
    if totalNrOfLines > 1000000:
        break

duration = round(time.time() - start,2)
print("\nbz2 file:"+ str(bz2Filename) )
print("Loading " + str(totalNrOfLines) +  " lines took: "  + str(duration) + " seconds")
print("Selected " + str(filteredNrOfLines) +  " lines")



bz2 file:../ping-2022-03-01T0000.bz2
Loading 1000001 lines took: 43.79 seconds
Selected 22516 lines


In [397]:
probes_set = set(probes_in_RIPE_list) #set probes in RIPE list

In [423]:
filtered_RIPE_df = pd.DataFrame(tuple_list, columns=('dst_addr', 'prb_id', 'avg')) #mapping dst_addr, prb_id and avg in a new dataframe

In [424]:
filtered_RIPE_df

,dst_addr,prb_id,avg
0,193.0.19.60,10603,34.819981
1,129.13.73.54,10603,44.838081
2,81.91.95.116,10603,45.931803
3,5.188.108.64,10603,18.466986
4,80.67.181.29,10603,38.368946
...,...,...,...
22511,182.161.74.241,6851,263.403971
22512,85.187.20.236,6851,67.888080
22513,18.228.126.142,6851,221.234405
22514,185.92.128.30,6851,137.254248


In [431]:
dictionary_ip=[]
for n in EU_country_list:
    
    temp=EU_ipv4_df[EU_ipv4_df['country_code']==n]
    ip_eu = list(zip(temp.ip_from, temp.ip_to))
    
    for i in range(filtered_RIPE_df.shape[0]):
                   
        if ip_exist(int(ipaddress.IPv4Address(filtered_RIPE_df['dst_addr'][i])),ip_eu):
                temp_dataset=pd.DataFrame(columns=['IP Address', 'Country_Code'])
                ip_add=int(ipaddress.IPv4Address(filtered_RIPE_df['dst_addr'][i]))
                country=n
                case={'ip_add': ip_add, 'country':n}
                dictionary_ip.append(case)
        else:
            pass
    

In [433]:
#Here are all the EU specific IP addresses 
#dictionary_ip

In [434]:
for n in EU_country_list:
    
    temp=EU_ipv4_df[EU_ipv4_df['country_code']==n]
    ip_eu = list(zip(temp.ip_from, temp.ip_to))
    p=0
    c=0
    
    for i in range(filtered_RIPE_df.shape[0]):
                   
        if ip_exist(int(ipaddress.IPv4Address(filtered_RIPE_df['dst_addr'][i])),ip_eu):
                p=p+filtered_RIPE_df['avg'][i]
                c=c+1
        else:
            pass
    if c!=0:
        print(str(n) +':' +str(p/c))
    else:
        print(str(n) + 'not found')
    
    

AT:89.20559681907709
BE:73.53399177422514
BG:99.9601853806574
HR:116.42880753535754
CYnot found
CZ:83.53638882076334
DK:89.84542445218564
EE:97.55639063636364
FI:84.70771428078864
FR:86.22186735900759
DE:83.9806181108743
GR:107.83651495995271
HU:107.88417809194824
IE:87.46615228000202
IT:85.19865502823502
LV:81.93910908200948
LT:98.34591091419112
LU:84.5238143070709
MTnot found
NL:73.55090694232969
PL:100.48087317070139
PT:122.69124855555496
RO:105.99527948148445
SKnot found
SI:98.97732270270272
ES:91.19195961027522
SE:83.19690695738521


In [435]:
filtered_RIPE_df.head() #show filtered RIPE with dst_addr, prb_id and avg columns

,prb_id,avg
count,22516.000000,22516.000000
mean,6650.103082,126.341176
std,594.230947,95.870864
min,2522.000000,0.043484
25%,6471.000000,38.857379
50%,6747.000000,104.669757
75%,6879.000000,200.932182
max,10678.000000,1100.504714


In [308]:
filtered_RIPE_df.isnull().any() #check wether there are null value in the dataset

dst_addr    False
prb_id      False
avg         False
dtype: bool

In [309]:
filtered_RIPE_df['dst_addr'].describe()

count              85
unique             74
top       92.60.13.86
freq                3
Name: dst_addr, dtype: object

In [310]:
filtered_RIPE_df['dst_addr'].min()

'129.13.73.54'

There are no 0.0.0.0 dst_addr which means the dataset for dst_addr is already valid

In [311]:
filtered_RIPE_df['prb_id'].describe()

count       85.000000
mean      7161.670588
std       1351.769849
min       6424.000000
25%       6424.000000
50%       6851.000000
75%       6852.000000
max      10664.000000
Name: prb_id, dtype: float64

Minimum and maximum value of prb_id is 2522 and 10678 respectively which means the dataset is valid

In [312]:
filtered_RIPE_df['avg'].describe()

count    85.000000
mean     31.822000
std      14.307192
min       3.063185
25%      20.833419
50%      31.650608
75%      41.008934
max      67.664701
Name: avg, dtype: float64

Minimum and maximum value of avg is 0.025559 and 1138.807999 which means the dataset is valid

In [313]:
#combine filtered_RIPE_df with sorted_df
#merge the AS dataset and Probe data
filtered_RIPE_df = filtered_RIPE_df.merge(sorted_df, left_on='prb_id', right_on='prb_id', how='inner')
filtered_RIPE_df = filtered_RIPE_df.replace(np.nan,'', regex=True) 

In [314]:
filtered_RIPE_df.head()

,dst_addr,prb_id,avg,ASN,Country,Name,NumIPs,type


In [315]:
#convert ipaddress from IPv4 format to integer format
import ipaddress
filtered_RIPE_df["dst_ipint"] = filtered_RIPE_df["dst_addr"].apply(lambda x:int(ipaddress.IPv4Address(x)))

In [316]:
filtered_RIPE_df.head() #show the new filtered_RIPE_df.head and now there is new column of IP with integer format

,dst_addr,prb_id,avg,ASN,Country,Name,NumIPs,type,dst_ipint


In [214]:
filtered_RIPE_df.count() #filtered_RIPE_df for EU country only since it is merged with AS dataset that already filtered by EU only

dst_addr     876
prb_id       876
avg          876
ASN          876
Country      876
Name         876
NumIPs       876
type         876
dst_ipint    876
dtype: int64

From total 648635 data in the 1 hour dataset, the data which already filtered by hosting type and used probe id (prb_id) is 22839 which is only 3.5% of total data in 1 hour.

Now, filter the data to identify data which target IPv4 is from an EU country only with reference to ipv4_df dataset.

In [215]:
#first filter ipv4_df for EU only
EU_ipv4_df = ipv4_df[ipv4_df.country_code.isin(EU_country_list)]
EU_ipv4_df.head(100)

,ip_from,ip_to,country_code,country_name
88,28499968,28508159,ES,Spain
96,28536832,28540927,FR,France
105,33554432,34603263,FR,France
106,34603264,34603519,PL,Poland
107,34603520,34603775,CZ,Czechia
...,...,...,...,...
246,34672128,34672383,PL,Poland
247,34672384,34673663,FR,France
252,34677248,34677503,FR,France
253,34677504,34677759,ES,Spain


In [216]:
EU_ipv4_df.count() #there are 65904 rows for ipv4 in EU only

ip_from         65904
ip_to           65904
country_code    65904
country_name    65904
dtype: int64

In [235]:
EU_ipv4_df.head(20000)

<class 'pandas.core.series.Series'>
Int64Index: 876 entries, 0 to 875
Series name: dst_ipint
Non-Null Count  Dtype
--------------  -----
876 non-null    int64
dtypes: int64(1)
memory usage: 13.7 KB


In [236]:
EUdf = EU_ipv4_df.head(100)
RIPEdf = filtered_RIPE_df.head(1000000)

In [237]:
len(EU_ipv4_df)

65904

In [ ]:
dict(zip(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']))

In [ ]:
numbers = filtered_RIPE_df['dst_ipint']
ranges = dict(zip(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to'])) 
score = {} # dict to store results in

for number in numbers:
    for range_name in sorted(ranges):
        range_list = ranges[range_name]
        groups = zip(*[iter(range_list)] * 2)
        if any(start <= number < end for start,end in groups):
            df3 = pd.DataFrame(numbers)
            #scores.setdefault(range_name, 0) += 1

In [ ]:
#x = 0
#for v in filtered_RIPE_df['dst_ipint']:
#    if x < len(EU_ipv4_df):
#        if v in range (EU_ipv4_df['ip_from'][x], EU_ipv4_df['ip_to'][x]):
#            df3 = True
#        else:
#            x = x + 1
#        if v in range(EU_ipv4_df['ip_from'][x], EU_ipv4_df['ip_to'][x]):
#         return True
#        else:
#         return False
#        x = x + 1
    #EU_ipv4_df = EU_ipv4_df[(EU_ipv4_df['ip_from'] < v) &  (EU_ipv4_df['ip_to'] > v)]

In [ ]:
EU_ipv4_df.count()

In [ ]:
filtered_RIPE_df[filtered_RIPE_df['dst_ipint'].isin(range(ip_range))]

In [ ]:
EU_ipv4_df['ip_range'] = list(zip(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']))

In [ ]:
EU_ipv4_df['ip_range'] = EU_ipv4_df.apply(range(EU_ipv4_df['ip_from'].astype(int), EU_ipv4_df['ip_to'].astype(int)))

In [ ]:
def check(filtered_RIPE_df['dst_ipint'], EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']): # check if i is in the range. # return True or False
    if filtered_RIPE_df['dst_ipint'] in range(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']):
        return True
    else:
        return False

In [ ]:
#dict(zip(df.state, df.name))
EU_ipv4_df['ip_range'] = dict(zip(EU_ipv4_df.ip_from, EU_ipv4_df.ip_to))

In [ ]:
a = filtered_RIPE_df['dst_addr'].reference.values
bh = EU_ipv4_df['ip_to'].end.values
bl = EU_ipv4_df['ip_from'].start.values

i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))

EU_df = pd.DataFrame(
    np.column_stack([filtered_RIPE_df['dst_addr'][i], EU_ipv4_df['ip_to'].values[j]]),
    columns=filtered_RIPE_df['dst_addr'].columns.append(EU_ipv4_df['ip_to'].columns)
)

# if you need not elements not in range

#filtered_RIPE_df.merge(EU_df, on='reference', how='left')

#filtered_RIPE_df['is_from_EU'] = filtered_RIPE_df['dst_ipint'].between(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to'])  
#filtered_RIPE_df['is_from_EU'] = filtered_RIPE_df['dst_ipint'] >= EU_ipv4_df['ip_from'] #& (filtered_RIPE_df['dst_ipint'] < EU_ipv4_df['ip_to'])

country n x probe id matrix -> average latency

d. Move from using only an hour to the full day. It is advisable to store the raw results of each file. Then,
using all processed files, calculate the average latency’s for each country-AS combination and store
the results into one ncountries ×m matrix. If we could place one server in each country, what would the
minimum average latency be for each country? (include in your report)

1. Combine all data from each hour (full day data)
2. Calcualte the average latency for each country - AS combination 
3. Store results into one n countries x m matrix (m = probe id) or ASN?
4. Find the minimum average latency for each country
(do visualization for the datasets)

e. Since we are only allowed to place four servers, determine the best four datacenters based on the total
latency for all countries. Report your findings and your procedure to obtain them. Also include the
average latency for each country.

1. Find the total latency for all countries * (?)
2. Find the average latency for each country
3. Sort/folter based on four datacenters (Question: datacenters is attached to the country or to the probe id?)
4. Average latency for each country.

5. Report findings
6. Report procedure to obtain them